# Querying Protests Worldwide
Requirements:
* ArcGIS API for Python
`conda install -c esri arcgis`
* arcpy or at least shapely for spatial joins


In [1]:
# author: Jan Tschada
# SPDX-License-Identifer: Apache-2.0

In [5]:
from arcgis.gis import GIS
from arcgis.features import FeatureSet
from arcgis.geometry.filters import intersects
from arcgis.mapping import generate_classbreaks
from datetime import datetime
from georapid.client import GeoRapidClient
from georapid.factory import EnvironmentClientFactory
from georapid.formats import OutFormat
from georapid.protests import aggregate
import pandas as pd

## Creating a client
---
**NOTE**

Ensure that `os.environ['x_rapidapi_key']` holds your Rapid API Key!

---

In [6]:
host = 'geoprotests.p.rapidapi.com'
client: GeoRapidClient = EnvironmentClientFactory.create_client_with_host(host)

## Connecting to ArcGIS Online

In [7]:
gis = GIS()

In [8]:
def generate_aggregated_renderer(spatial_df, column='count'):
    alpha = 0.35
    class_breaks_renderer = generate_classbreaks(spatial_df, geometry_type='Polygon', field=column, method='esriClassifyNaturalBreaks', class_count=5, colors='YlOrRd', alpha=0.35)
    class_breaks_renderer['defaultSymbol']['color'][3] = int(alpha * 255)
    for class_break_info in class_breaks_renderer['classBreakInfos']:
        class_break_info['symbol']['color'][3] = int(alpha * 255)
    return class_breaks_renderer

def plot_aggregated(map_view, spatial_df, column='count'):
    """
    Plots the spatial dataframe as classified polygons using the specified map view.
    """
    if spatial_df.empty:
        print("The dataframe is empty!")
    else:
        class_breaks_renderer = generate_aggregated_renderer(spatial_df)
        drawing_info = {
            'renderer': class_breaks_renderer
        }
        feature_collection = spatial_df.spatial.to_feature_collection(drawing_info=drawing_info)
        map_view.add_layer(feature_collection)
        
def query_aggregations(from_date, to_date):
    """
    Queries the specific date range and returns a data frame.
    """
    sdfs = []
    date_range = pd.date_range(start=from_date, end=to_date)
    for date_of_interest in date_range:
        aggregated_protests_dict = aggregate(client, date_of_interest, OutFormat.ESRI)
        aggregated_protests_fset = FeatureSet.from_dict(aggregated_protests_dict)
        aggregated_protests_sdf = aggregated_protests_fset.sdf
        if not aggregated_protests_sdf.empty:
            sdfs.append(aggregated_protests_sdf)
            
    if sdfs:
        return pd.concat(sdfs, ignore_index=True)
    else:
        return pd.DataFrame()
        
def query_world_countries(gis, two_digit_country_code):
    """
    Returns the geographic feature of a country.
    """
    world_countries_item_id = '2b93b06dc0dc4e809d3c8db5cb96ba69'
    world_countries_item = gis.content.get(world_countries_item_id)
    world_countries_layer = world_countries_item.layers[0]
    return world_countries_layer.query(where=f"ISO='{two_digit_country_code}'")

## Query the protests and try to narrow down those related to the French pension reform strikes

In [9]:
protest_map = gis.map('France')
protest_map.basemap = 'osm'
protest_map

MapView(layout=Layout(height='400px', width='100%'))

In [10]:
aggregated_protests_sdf = query_aggregations(datetime(2023, 1, 19), datetime(2023, 1, 29))
aggregated_protests_sdf.sort_values(by='count', ascending=False)

,OBJECTID,count,timestamp,Shape__Area,Shape__Length,SHAPE
18,38893,1028,2023-01-19,54126587736.526367,866025.403784,"{""rings"": [[[-8779178.09360229, -1337491.6572]..."
220,39095,793,2023-01-21,54126587736.526367,866025.403784,"{""rings"": [[[-8779178.09360229, -1337491.6572]..."
113,38988,722,2023-01-20,54126587736.526367,866025.403784,"{""rings"": [[[-8779178.09360229, -1337491.6572]..."
997,39872,614,2023-01-29,54126587736.527344,866025.403784,"{""rings"": [[[-8346165.39171007, 4912508.3428],..."
315,39190,594,2023-01-22,54126587736.523438,866025.403784,"{""rings"": [[[-9428697.14644062, 4037508.3428],..."
...,...,...,...,...,...,...
506,39381,1,2023-01-24,54126587736.527344,866025.403784,"{""rings"": [[[-8562671.74265618, -1712491.6572]..."
505,39380,1,2023-01-24,54126587736.526367,866025.403784,"{""rings"": [[[-8562671.74265618, -1462491.6572]..."
504,39379,1,2023-01-24,54126587736.529297,866025.403784,"{""rings"": [[[-8562671.74265618, -1212491.6572]..."
502,39377,1,2023-01-24,54126587736.527588,866025.403784,"{""rings"": [[[-8562671.74265618, 287508.3427999..."


In [11]:
protest_map = gis.map('France')
protest_map.basemap = 'osm'
plot_aggregated(protest_map, aggregated_protests_sdf)
protest_map

MapView(layout=Layout(height='400px', width='100%'))

## Filter the news related to protests being located in France
---
**NOTE**

If your environment has no geometry engine (`arcpy` or `shapely`) the spatial join returns an empty data frame!

---

In [12]:
france_country_fset = query_world_countries(gis, two_digit_country_code='FR')
france_country_sdf = france_country_fset.sdf
france_country_sdf

,FID,COUNTRY,ISO,COUNTRYAFF,AFF_ISO,Shape__Area,Shape__Length,SHAPE
0,79,France,FR,France,FR,1162358729551.715332,8568685.86359,"{""rings"": [[[198339.953048288, 5246743.3982777..."


In [13]:
france_aggregated_protests_sdf = aggregated_protests_sdf.spatial.join(france_country_sdf[['COUNTRY', 'SHAPE']])
france_aggregated_protests_sdf = france_aggregated_protests_sdf.drop(columns=['index_right'])
france_aggregated_protests_sdf

,OBJECTID,count,timestamp,Shape__Area,Shape__Length,SHAPE,COUNTRY
0,38916,9,2023-01-19,54126587736.52771,866025.403784,"{""rings"": [[[-551936.757650122, 6162508.3428],...",France
1,38920,1,2023-01-19,54126587736.527435,866025.403784,"{""rings"": [[[-118924.055757903, 6412508.3428],...",France
2,38921,5,2023-01-19,54126587736.527527,866025.403784,"{""rings"": [[[-118924.055757903, 5912508.3428],...",France
3,38922,5,2023-01-19,54126587736.527405,866025.403784,"{""rings"": [[[-118924.055757903, 5662508.3428],...",France
4,38923,4,2023-01-19,54126587736.527344,866025.403784,"{""rings"": [[[-118924.055757903, 5412508.3428],...",France
...,...,...,...,...,...,...,...
60,39804,2,2023-01-28,54126587736.527344,866025.403784,"{""rings"": [[[-118924.055757903, 5412508.3428],...",France
61,39805,1,2023-01-28,54126587736.527283,866025.403784,"{""rings"": [[[97582.2951882072, 6537508.3428], ...",France
62,39806,6,2023-01-28,54126587736.527466,866025.403784,"{""rings"": [[[97582.2951882072, 6287508.3428], ...",France
63,39809,2,2023-01-28,54126587736.527344,866025.403784,"{""rings"": [[[314088.646134317, 5662508.3428], ...",France


In [14]:
protest_map = gis.map('France')
protest_map.basemap = 'osm'
plot_aggregated(protest_map, france_aggregated_protests_sdf)
protest_map

MapView(layout=Layout(height='400px', width='100%'))